## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked', 'PerformanceRating', 'RelationshipSatisfaction', 'Department', 'Attrition']

# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes

Age                          int64
DistanceFromHome             int64
Education                    int64
EnvironmentSatisfaction      int64
JobInvolvement               int64
JobLevel                     int64
JobSatisfaction              int64
NumCompaniesWorked           int64
PerformanceRating            int64
RelationshipSatisfaction     int64
Department                  object
Attrition                   object
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df = pd.get_dummies(X_df, drop_first=True)
X_df.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PerformanceRating,RelationshipSatisfaction,Department_Research & Development,Department_Sales,Attrition_Yes
0,41,1,2,2,3,2,4,8,3,1,False,True,True
1,49,8,1,3,2,2,2,1,4,4,True,False,False
2,37,2,2,4,2,1,3,6,3,2,True,False,True
3,33,3,4,4,3,1,3,1,3,3,True,False,False
4,27,2,1,1,3,1,2,9,3,4,True,False,False


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

# Scale the training and testing data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_d = encoder.transform(y_train[['Department']])
y_test_encoded_d = encoder.transform(y_test[['Department']])

In [9]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_a = encoder.transform(y_train[['Attrition']])
y_test_encoded_a = encoder.transform(y_test[['Attrition']])



## Part 2: Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data.
num_cols = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_cols,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(10, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(5, activation='relu')(shared_layer_1)


In [11]:
# Create a branch for Department
# with a hidden layer and an output layer
department_branch = layers.Dense(32, activation='relu')(shared_layer_2)
# Create the hidden layer
department_branch = layers.Dense(16, activation='relu')(department_branch)

# Create the output layer
department_output = layers.Dense(y_train_encoded_d.shape[1], activation='softmax', name='Department')(department_branch)

In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_branch = layers.Dense(32, activation='relu')(shared_layer_2)
# Create the hidden layer
attrition_branch = layers.Dense(16, activation='relu')(attrition_branch)

# Create the output layer
attrition_output = layers.Dense(y_train_encoded_a.shape[1], activation='softmax', name='Attrition')(attrition_branch)

In [15]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'Department': 'categorical_crossentropy', 'Attrition': 'categorical_crossentropy'},
              )

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 10)        │        140 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5)         │         55 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │        192 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │        192 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 16)        │        528 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Department (Dense)  │ (None, 3)         │         51 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Attrition (Dense)   │ (None, 2)         │         34 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,720 (6.72 KB)

 Trainable params: 1,720 (6.72 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
history = model.fit(X_train, {'Department': y_train_encoded_d, 'Attrition': y_train_encoded_a},
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - Attrition_loss: 0.6265 - Department_loss: 1.1100 - loss: 1.7368 - val_Attrition_loss: 0.5289 - val_Department_loss: 0.8723 - val_loss: 1.4021
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Attrition_loss: 0.4828 - Department_loss: 0.8289 - loss: 1.3118 - val_Attrition_loss: 0.4687 - val_Department_loss: 0.6247 - val_loss: 1.0953
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Attrition_loss: 0.4368 - Department_loss: 0.5961 - loss: 1.0332 - val_Attrition_loss: 0.4355 - val_Department_loss: 0.4082 - val_loss: 0.8460
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Attrition_loss: 0.4086 - Department_loss: 0.3813 - loss: 0.7902 - val_Attrition_loss: 0.4074 - val_Department_loss: 0.2687 - val_loss: 0.6783
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Attrition_loss: 0.3558 - Department_loss: 0.2308 - loss: 0.5864 - val_Attrition_loss: 0.3789 - val_Department_loss: 0.2038 - val_loss: 0.5846
Epoch 6/50
28/28 ━━━━━━━━━━━━━

In [22]:
# Evaluate the model with the testing data
results = model.evaluate(X_test, {'Department': y_test_encoded_d, 'Attrition': y_test_encoded_a}, verbose=1)





12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Attrition_loss: 2.8009e-04 - Department_loss: 0.0026 - loss: 0.0029 


In [25]:
# Print the accuracy for both department and attrition as a percentage
print(f"Department Accuracy: {results[1] * 100:.2f}%")
print(f"Attrition Accuracy: {results[2] * 100:.2f}%")


Department Accuracy: 0.17%
Attrition Accuracy: 0.02%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 